# Pregunta 3

Debido al cruce entre recursos de las preguntas, se prefirió usar el formato Jupyer Notebook para el desarrollo de esta pregunta

## Librerias usadas

In [3]:
import numpy as np
import pandas as pd

## Item a)

### Funciones

In [4]:
def couple(pair):
    """
    Verifica si un par de nucleótidos corresponde a pares de bases complementarias (Watson-Crick).

    Args:
        pair (tuple): Una tupla conteniendo dos caracteres (nucleótidos), ej. ('A', 'U').

    Returns:
        bool: True si son complementarios (A-U, G-C, U-A, C-G), False en caso contrario.
    """
    pairs = {"A": "U", "U": "A", "G": "C", "C": "G"}  # Bases complementarias

    # Revisar si bases son complementarias
    if pair in pairs.items():
        return True

    return False

In [5]:
def fill(nm, rna):
    """
    Rellena la matriz de programación dinámica de Nussinov siguiendo las 4 reglas del algoritmo.

    Esta función itera sobre la matriz calculando los puntajes máximos de apareamiento para
    subsecuencias de longitud creciente (k).

    Args:
        nm (numpy.ndarray): Matriz inicializada (generalmente con NaNs).
        rna (str): La secuencia de ARN a procesar.

    Returns:
        numpy.ndarray: La matriz llena con los puntajes de la estructura óptima.
    """
    minimal_loop_length = 0
    n_bases = len(rna)
    for k in range(1, n_bases):
        for i in range(n_bases - k):
            j = i + k

            if j - i >= minimal_loop_length:
                if (i == 0) and (j == n_bases - 1):
                    down = nm[i + 1][j]  # 1st rule
                    left = nm[i][j - 1]  # 2nd rule
                    # diag = nm[i + 1][j - 1] + couple((rna[i], rna[j]))  # Se elimina la tercera regla para el caso de la última revisión, para evitar parear la primera y última base

                    rc = max([nm[i][t] + nm[t + 1][j] for t in range(i, j)])  # 4th rule

                    nm[i][j] = max(down, left, rc)  # max of all
                else:
                    down = nm[i + 1][j]  # 1st rule
                    left = nm[i][j - 1]  # 2nd rule
                    diag = nm[i + 1][j - 1] + couple((rna[i], rna[j]))  # 3rd rule

                    rc = max([nm[i][t] + nm[t + 1][j] for t in range(i, j)])  # 4th rule

                    nm[i][j] = max(down, left, diag, rc)  # max of all

            else:
                nm[i][j] = 0

    return nm

In [6]:
def traceback(nm, rna, fold, i, L):
    """
    Recorre recursivamente la matriz de Nussinov completa para encontrar la solución óptima.

    Realiza el backtracking desde la esquina superior derecha de la matriz hacia la diagonal,
    determinando qué regla originó el puntaje actual para reconstruir los pares.

    Args:
        nm (numpy.ndarray): La matriz de Nussinov llena.
        rna (str): La secuencia de ARN.
        fold (list): Lista para acumular las tuplas de índices apareados (i, j).
        i (int): Índice de inicio actual.
        L (int): Índice de fin actual (j).

    Returns:
        list: Lista de tuplas con los índices de las bases apareadas.
    """
    j = L

    if i < j:
        if nm[i][j] == nm[i + 1][j]:  # 1st rule
            traceback(nm, rna, fold, i + 1, j)
        elif nm[i][j] == nm[i][j - 1]:  # 2nd rule
            traceback(nm, rna, fold, i, j - 1)
        elif nm[i][j] == nm[i + 1][j - 1] + couple((rna[i], rna[j])):  # 3rd rule
            fold.append((i, j))
            traceback(nm, rna, fold, i + 1, j - 1)
        else:
            for k in range(i + 1, j - 1):
                if nm[i][j] == nm[i, k] + nm[k + 1][j]:  # 4th rule
                    traceback(nm, rna, fold, i, k)
                    traceback(nm, rna, fold, k + 1, j)
                    break

    return fold

In [7]:
def dot_write(rna, fold):
    """
    Genera la representación de la estructura secundaria en formato 'dot-bracket'.

    Convierte la lista de pares de bases en una cadena donde '.' es una base no apareada,
    '(' es una base de inicio de par y ')' es una base de cierre.

    Args:
        rna (str): La secuencia de ARN original (usada para determinar la longitud).
        fold (list): Lista de tuplas con los índices apareados.

    Returns:
        str: Cadena de texto con la estructura secundaria (ej. "((...))").
    """
    dot = ["." for i in range(len(rna))]

    for s in fold:
        # print(min(s), max(s))
        dot[min(s)] = "("
        dot[max(s)] = ")"

    return "".join(dot)

In [8]:
def init_matrix(rna):
    """
    Inicializa la matriz cuadrada NxN para el algoritmo.

    Crea una matriz llena de NaNs y establece la diagonal principal y la adyacente inferior en 0,
    condición base para la programación dinámica.

    Args:
        rna (str): La secuencia de ARN.

    Returns:
        numpy.ndarray: Matriz inicializada lista para ser llenada.
    """
    M = len(rna)

    # Inicializa la matriz
    nm = np.empty([M, M])
    nm[:] = np.nan

    # Inicializa las diagonales a 0
    # varias formas de hacer esto: np.fill_diagonal(), np.diag(), bucle anidado, ...
    nm[range(M), range(M)] = 0
    nm[range(1, M), range(M - 1)] = 0

    return nm

In [9]:
def algoritmo_nussinov(rna: str) -> str:
    """
    Ejecuta el algoritmo de Nussinov para predecir la estructura secundaria de una secuencia de ARN.

    Args:
        rna (str): Secuencia de ARN.

    Returns:
        str: Estructura secundaria en formato dot-bracket.
        numpy.ndarray: Matriz de Nussinov completa.
    """
    nm = init_matrix(rna)
    nm = fill(nm, rna)

    fold = []
    traceback(nm, rna, fold, 0, len(rna) - 1)

    res = dot_write(rna, fold)
    return [res, nm]

In [10]:
def view_matrix(nm: np.ndarray, rna: str) -> pd.DataFrame:
    """
    Crea un DataFrame de pandas para visualizar la matriz de Nussinov.

    Args:
        nm (np.ndarray): Matriz de Nussinov.
        rna (str): Secuencia de ARN.

    Returns:
        pd.DataFrame: DataFrame con la matriz y etiquetas de filas/columnas.
    """
    names = list(rna)
    df = pd.DataFrame(nm, index=names, columns=names)
    return df

### Resolución de la pregunta


Código basado en el desarrollo original de Carlos G. Oliver

Código originalmente implementado por Carlos G. Oliver: [https://github.com/cgoliver/Nussinov/blob/master/nussinov.py](https://github.com/cgoliver/Nussinov/blob/master/nussinov.py) modificado para esta pregunta de la tarea. 

In [12]:
rna = input("Ingrese la secuencia de ARN: ")  # Ejemplo: "AUUCUAG"
secuencia_parentesis, matriz_nussinov = algoritmo_nussinov(rna)
df = view_matrix(matriz_nussinov, rna)
print(df)
print(f"Secuencia de parentesis: {secuencia_parentesis}")
print(f"Maxima cantidad de pares: {matriz_nussinov[0][-1]}")

     A    U    U    C    U    A    G
A  0.0  1.0  1.0  1.0  1.0  2.0  3.0
U  0.0  0.0  0.0  0.0  0.0  1.0  2.0
U  NaN  0.0  0.0  0.0  0.0  1.0  2.0
C  NaN  NaN  0.0  0.0  0.0  1.0  2.0
U  NaN  NaN  NaN  0.0  0.0  1.0  1.0
A  NaN  NaN  NaN  NaN  0.0  0.0  0.0
G  NaN  NaN  NaN  NaN  NaN  0.0  0.0
Secuencia de parentesis: ().(())
Maxima cantidad de pares: 3.0


## Item b)

### Funciones usadas

In [ ]:
def nussinov_circular(secuencia: str):
    """
    Predice la estructura secundaria de un ARN circular aplicando el algoritmo de Nussinov
    a todas las posibles permutaciones cíclicas (linearización).

    El algoritmo considera que el ARN es un círculo cerrado. Para resolverlo con Nussinov (lineal),
    se corta el círculo en cada posición posible, generando 'n' secuencias lineales.
    Se asume que la versión lineal de Nussinov utilizada impide el apareamiento entre
    el primer y último nucleótido (extremos), ya que en la forma circular son vecinos.

    Args:
        secuencia (str): La secuencia de nucleótidos del ARN circular (ej: "AUUCUAG").

    Returns:
        List[str]: Una lista conteniendo:
            - [0]: La secuencia rotada que produjo la mejor estructura.
            - [1]: La estructura secundaria en formato dot-bracket correspondiente.
    """
    n = len(secuencia)
    rotaciones = [secuencia[i:] + secuencia[:i] for i in range(n)]
    max_pares = 0
    mejor_rotacion = 0
    mejor_estructura = ""
    mejor_seq = ""
    for i, seq_rotada in enumerate(rotaciones):
        estructura, matriz = algoritmo_nussinov(seq_rotada)
        cant_pares = matriz[0][-1]
        # print(
        #     f"Rotación {i}: \n\tS. Rotada: {seq_rotada}\n\tS. parentesis balanceados:  {estructura}\n\tMaxima cantidad de pares: {cant_pares}\n"
        # )
        if cant_pares > max_pares:
            max_pares = cant_pares
            mejor_rotacion = i
            mejor_estructura = estructura
            mejor_seq = seq_rotada
    print(
        f"Mejor resultado:\n\tRotación:{mejor_rotacion} ({mejor_seq})\n\tCantidad de pares: {max_pares}\n\tSecuencia de parentesis balanceados: {mejor_estructura}"
    )
    return [mejor_seq, mejor_estructura]

### Resolución de la pregunta

In [22]:
secuencia = input("Ingrese la secuencia de ARN: ")  # Ejemplo AUUCUAG
secuencia_circular, estructura_circular = nussinov_circular(secuencia)

Mejor resultado:
	Rotación:0 (AUUCUAG)
	Cantidad de pares: 3.0
	Estructura secundaria: ().(())


## Item c)

In [24]:
seq_og = "AUGGUACUGCCAUUCAAGAUGA"
nussinov_circular(seq_og)

Mejor resultado:
	Rotación:0 (AUGGUACUGCCAUUCAAGAUGA)
	Cantidad de pares: 9.0
	Estructura secundaria: ().(())(.((.((.)))()))


['AUGGUACUGCCAUUCAAGAUGA', '().(())(.((.((.)))()))']

In [25]:
seq_alt = "UCGGUAUGCCAAUACCCGUAAG"
nussinov_circular(seq_alt)

Mejor resultado:
	Rotación:0 (UCGGUAUGCCAAUACCCGUAAG)
	Cantidad de pares: 9.0
	Estructura secundaria: .(((()(().)().))()().)


['UCGGUAUGCCAAUACCCGUAAG', '.(((()(().)().))()().)']